# 빙산인가? 선박인가? - 3

# 데이터 만들기

In [3]:
import numpy as np
import pandas as pd
import os

# 캐글 노트북에서 진행할 경우, 4.3.1절을 참고하세요.
DATA_PATH = './data/'

train = pd.read_json(os.path.join(DATA_PATH, 'train.json'))
test = pd.read_json(os.path.join(DATA_PATH, 'test.json'))

# (75, 75)의 형태로 변경해줍니다.
band_1 = np.array([np.array(band).reshape(75, 75) for band in train['band_1']])
band_2 = np.array([np.array(band).reshape(75, 75) for band in train['band_2']])

def get_mean_std(array):
    return np.mean(array), np.std(array)

band_1_mean, band_1_std = get_mean_std(band_1)
band_2_mean, band_2_std = get_mean_std(band_2)

# 각 이미지별로 표준화를 적용합니다.
band_1_st = (band_1 - band_1_mean) / band_1_std
band_2_st = (band_2 - band_2_mean) / band_2_std

x_train = np.concatenate([band_1_st[:, :, :, np.newaxis], band_2_st[:, :, :, np.newaxis],
                          ((band_1_st+band_2_st)/2)[:, :, :, np.newaxis]], axis=-1)
train['inc_angle'] = train['inc_angle'].replace('na', 0)
x_angle_train = np.array(train['inc_angle'])

y_train = np.array(train['is_iceberg'])

from sklearn.model_selection import train_test_split

# 훈련/검증 데이터를 0.7/0.3의 비율로 분리합니다.
x_train, x_val, x_angle_train, x_angle_val, y_train, y_val = train_test_split(x_train, x_angle_train, y_train, 
                                                                              test_size = 0.2, random_state = 777)

# 평가에 사용할 테스트 데이터를 만듭니다.
test_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
test_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

test_band_1_st = (test_band_1 - band_1_mean) / band_1_std
test_band_2_st = (test_band_2 - band_2_mean) / band_2_std

x_test = np.concatenate([test_band_1_st[:, :, :, np.newaxis]
                          , test_band_2_st[:, :, :, np.newaxis]
                         , ((test_band_1_st+test_band_2_st)/2)[:, :, :, np.newaxis]], axis=-1)
x_angle_test = np.array(test['inc_angle'])


print('훈련 데이터 {} 레이블 {}'.format(str(x_train.shape), str(y_train.shape)))
print('검증 데이터 {} 레이블 {}'.format(str(x_val.shape), str(y_val.shape)))
print('입사각 훈련 데이터 {} 검증 데이터 {}'.format(str(x_angle_train.shape), str(x_angle_val.shape)))
print('테스트 데이터 {} 입사각 테스트 데이터 {}'.format(str(x_test.shape), str(x_angle_test.shape)))

훈련 데이터 (1283, 75, 75, 3) 레이블 (1283,)
검증 데이터 (321, 75, 75, 3) 레이블 (321,)
입사각 훈련 데이터 (1283,) 검증 데이터 (321,)
테스트 데이터 (8424, 75, 75, 3) 입사각 테스트 데이터 (8424,)


# 모델 구성하기

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Input
from tensorflow.keras.models import Model

def get_model():
    # 이미지 입력
    img_input = Input(shape=(75, 75, 3), name="img")
    # 입사각 입력
    angle_input = Input(shape=[1], name="angle")
    
    # 이미지의 입력을 다룹니다.
    base_model = ResNet50(weights = 'imagenet', include_top = False, input_shape = (75, 75, 3))(img_input)
    img_x = GlobalAveragePooling2D()(base_model)
    img_x = Dense(512, activation = 'relu')(img_x)
    img_x = Dropout(0.5)(img_x)
    img_x = Dense(256, activation = 'relu')(img_x)
    img_x = Dropout(0.5)(img_x)
    img_x = Dense(128, activation = 'relu')(img_x)
    img_x = Dropout(0.3)(img_x)
    
    # 입사각의 입력을 이미지를 분석한 층과 합칩니다.
    angle_concat = Concatenate()([img_x, angle_input])
    
    x = Dense(64, activation = 'relu')(angle_concat)
    x = Dropout(0.3)(x)
    x = Dense(32, activation = 'relu')(x)
    x = Dropout(0.3)(x)
    # 출력값을 만듭니다. 
    output = Dense(1, activation = 'sigmoid')(x)
    
    model = Model(inputs = [img_input, angle_input], outputs = output)
    model.compile(optimizer = 'adam',
                 loss = 'binary_crossentropy',
                 metrics = ['acc'])

    return model

model = get_model()
print('model ready ~')

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 4d473c1dd8becc155b73f8504c6f6626 so we will re-download the data.
94773248/94765736 [==============================] - 25s 0us/step
model ready ~


# 학습하기

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

filepath = './model'

if(not os.path.exists(filepath)):
    os.mkdir(filepath)

callbacks = [ModelCheckpoint(filepath + '/best_model.hdf5', save_best_only = True, verbose = 1),
            ReduceLROnPlateau(monitor = 'val_loss', patience = 5)]

model.fit([x_train, x_angle_train], y_train, epochs = 30,
         validation_data = ([x_val, x_angle_val], y_val),
         batch_size = 32,
         callbacks = callbacks)

Train on 1283 samples, validate on 321 samples
Epoch 1/30
1283/1283 [==============================] - 30s 24ms/sample - loss: 0.8628 - acc: 0.5916 - val_loss: 54582.2854 - val_acc: 0.5202
Epoch 2/30
1283/1283 [==============================] - 9s 7ms/sample - loss: 0.6542 - acc: 0.6399 - val_loss: 123.6341 - val_acc: 0.5202
Epoch 3/30
1283/1283 [==============================] - 9s 7ms/sample - loss: 0.5082 - acc: 0.7428 - val_loss: 0.6501 - val_acc: 0.5732
Epoch 4/30
1283/1283 [==============================] - 8s 6ms/sample - loss: 0.4426 - acc: 0.8153 - val_loss: 77.3480 - val_acc: 0.5202
Epoch 5/30
1283/1283 [==============================] - 8s 6ms/sample - loss: 0.4114 - acc: 0.8239 - val_loss: 495.5134 - val_acc: 0.5202
Epoch 6/30
1283/1283 [==============================] - 8s 6ms/sample - loss: 0.3247 - acc: 0.8730 - val_loss: 159.7382 - val_acc: 0.5202
Epoch 7/30
1283/1283 [==============================] - 8s 6ms/sample - loss: 0.3982 - acc: 0.8644 - val_loss: 0.6748 - val_

# 모델의 저장과 복원

In [ ]:
model.save('./path.hdf5') # 모델을 저장합니다.

model.load_weights('./path.hdf5') # 모델의 가중치만 볼러옵니다.

model.load_model('./path.hdf5') # 모델의 구조와 가중치를 불러옵니다.

# 결과 만들기

In [8]:
# 모델의 가중치만 불러옵니다.
model.load_weights(filepath + '/best_model.hdf5')

predicted_test = model.predict([x_test, x_angle_test])

# 캐글에 제출할 형식을 만듭니다.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)